In [179]:
import os, re
import pypandoc
import numpy as np
import pandas as pd
from pypandoc.pandoc_download import download_pandoc
from collections import OrderedDict

In [88]:
filepaths = []
for filename in os.listdir('../Корпус пар/'):
    if filename.endswith('.docx'):
        filepaths.append('../Корпус пар/'+filename)

In [91]:
df = pd.DataFrame(columns = ['id', 'transcription', 'gloss', 'translation', 'start', 'end', 'comment'])

for docxFilename in filepaths:
    print(docxFilename)
    output = pypandoc.convert_file(docxFilename, 'plain', outputfile="somefile.txt")
    assert output == ""
    
    with open('somefile.txt') as f:
        text = f.read()
    text = text.replace('\t', ' ')
    pattern_exmpl = r'([,A-Za-z]+_\d+@[,A-Za-z]+_\d+(\n|.)+?\d{2}:\d{2}:\d{2}\.\d{3} — \d{2}:\d{2}:\d{2}\.\d{3}.*?\n)'
    for finded in re.findall(pattern_exmpl, text):
        example = finded[0].split('\n\n')
        len_exmpl = len(example)
        
        timecodes_comment = example[-1].split(' ', 3)
        if len(timecodes_comment) > 3:
            comment = timecodes_comment[3].strip()
        else:
            comment = ''
            
        if len_exmpl in [3, 5, 7, 9]:
            if len_exmpl == 3:
                transcription = ''
                gloss = ''
                
            if len_exmpl == 7:
                transcription = (example[1].strip()+' '+example[3].strip()).replace('\n', ' ').replace('  ', ' ')
                gloss = (example[2].strip()+' '+example[4].strip()).replace('\n', ' ').replace('  ', ' ')        
            elif len_exmpl == 5:
                transcription = example[1].replace('\n', ' ').strip()
                gloss = example[2].replace('\n', ' ').strip()
            if len_exmpl == 3:
                transcription = ''
                gloss = ''
    
            to_add_to_df = [example[0],
                            transcription.lower(),
                            gloss.lower(),
                            example[-2].strip().strip('‘’'),
                            timecodes_comment[0],
                            timecodes_comment[2].strip(),
                            comment]
            df.loc[len(df)] = to_add_to_df
        else:
            print(len_exmpl)
            print(example)
            print('---')

../Корпус пар/ckt_TTI_20230703_TK,SZ_TK.docx
../Корпус пар/ckt_TVI_20230713_TK.docx
../Корпус пар/ckt_DMP_20230714_TK,AV_AV.docx
../Корпус пар/ckt_RGD,RDA_20230705_PK,TK_TK.docx
4
['RGD,RDA_20230705@TK_32', '  Мини дз', '  ‘Лукашенко говорит: "Не могут без меня картошку собрать".’', '00:52:42.200 — 00:52:57.676\n']
---
../Корпус пар/ckt_RGD_20230708_LBa,TK_TK.docx
4
['RGD_20230708@TK_31', '  минитекст', '  ‘’', '00:54:04.220 — 00:54:09.420 Про человека, который неожиданно знал\n']
---
4
['RGD_20230708@TK_32', '  минитекст', '  ‘’', '00:54:34.650 — 00:54:41.000\n']
---
../Корпус пар/ckt_KTV_20230702_AI,TK_TK.docx
../Корпус пар/ckt_KTV_20230710_TK_TK.docx
4
['KTV_20230710@TK_37', '  -', '  ‘’', '00:58:01.790 — 00:58:12.032 Почему собаку DMP так зовут.\n']
---
../Корпус пар/ckt_RGD_202200807_AV,TK_TK.docx
../Корпус пар/ckt_ENP_20230717_TK,AV_TK.docx
../Корпус пар/ckt_EIX_20230707_AI,TK_TK.docx
../Корпус пар/ckt_EMS_20230711_PK,TK_TK.docx
../Корпус пар/ckt_DMP_20230714_TK,AV_TK.docx
../Кор

In [208]:
patterns = OrderedDict({r'com-([^ ]*?)-sim':'com-...-sim',
            r'com-([^ ]*?)-ins':'com-...-ins',
            r'([^ ]*?)-attr':'attr',
            r'car-([^ ]*?)-car':'car',
            'вместе()':'вместе',
            r'([^ ]*?)=с':'reen',
            r'([^ ]*?)=вместе':'reen',
            ' и (.*?) ':'и'})

def get_com_type(gloss):
    com_type = ''
    for pattern in patterns:
        if re.search(pattern, gloss):
          com_type = patterns[pattern]
    return com_type
def get_companion(gloss):
    companion = ''
    for pattern in patterns:
        if re.search(pattern, gloss):
          companion = re.search(pattern, gloss).group(1)
    return companion
def get_apart(gloss):
    cands_apart = re.findall(r'([^ ]*?)-(an\.)?ins', gloss)
    for cand in cands_apart:
        if 'com' not in cand[0] and 'car' not in cand[0]:
            return cand[0]
    return ''

In [209]:
df['author'] = df['id'].apply(lambda x: x.split('_')[0])
df['binom'] = df['comment'].apply(lambda x: True if '#бином' in x else '')
df['notbinom'] = df['comment'].apply(lambda x: True if ('#небином' in x) or ('#INS_NOM' in x) else '')
df['*binom'] = df['comment'].apply(lambda x: True if '#запретнабином' in x else '')
df['*notbinom'] = df['comment'].apply(lambda x: True if '#запретнанебином' in x else '')
df['com_type'] = df['gloss'].apply(get_com_type)
df['companion'] = df['gloss'].apply(get_companion)
df['ins-participant'] = df['gloss'].apply(get_apart)

In [210]:
df = df.drop(np.where(df['transcription'] == '')[0])

In [211]:
df.to_csv('ckt_comitative_db.csv')

In [212]:
df.to_excel('ckt_comitative_db.xlsx')

In [213]:
df

,id,transcription,gloss,translation,start,end,comment,author,binom,notbinom,*binom,*notbinom,com_type,companion,ins-participant
0,TTI_20230703@TK_1,sas̆a ətri tanʲa ne-re-keɬi-ŋe-n keɬikeɬ,саша и таня low.a-fut-писать-fut-3sg.o книга,Саша и Таня напишут книжку.,00:01:20.930,00:01:27.590,#бином Я предложила sas̆a-na ətri tanʲa-na.,TTI,True,,,,и,таня,
1,TTI_20230703@TK_2,ne-re-keɬi-ŋe-n keɬikeɬ sas̆a ətri tanʲa,low.a-fut-писать-fut-3sg.o книга саша и таня,Напишут книжку Саша и Таня.,00:01:49.060,00:01:54.210,#бином Я предложила sas̆a-na ətri tanʲa-na.,TTI,True,,,,,,
2,TTI_20230703@TK_3,sas̆a ətri tanʲa ərɣə-nan ne-re-keɬi-ŋe-n keɬikeɬ,саша и таня они-ins low.a-fut-писать-fut-3sg.o...,Саша и Таня напишут книжку.,00:02:20.980,00:02:27.100,#бином #восстановление Я (породив): хорошо?,TTI,True,,,,и,таня,они
3,TTI_20230703@TK_4,mikə-ne ∅-re-keɬi-ɣ-ni-n keɬikeɬ? (sas̆a ətri ...,кто-an.ins 2/3.s/a-fut-писать-fut-3sg.a.3sg.o-...,Кто напишет книжку? Саша и Таня напишут книжку.,00:03:18.970,00:03:39.400,"Сказала целиком с биномом, потом ""А!"" и",TTI,,,,,и,таня,кто
4,TTI_20230703@TK_5,əɬɣu ɬəŋ-jo-tte kenti-t,любимый считать-ptcp.pass-pl конфета-pl,Любимые конфеты,00:04:56.630,00:05:02.130,,TTI,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,DMP_20220809@TK_27,ŋe-ekke-qej-e (*/ŋe-ekke-qej-∅) na-n-sarejpa-u...,fem-сын-dim-ins fem-сын-dim-nom.sg low.a-tr-па...,Девочка с другом испачкали рубашку.,00:37:10.190,00:37:17.507,#INS_NOM,DMP,,True,,,com-...-ins,друг,fem-сын-dim
636,DMP_20220809@TK_28,ŋe-ekke-qej-∅ ∅-r-sarejpa-u-ni-n məsəkwən,fem-сын-dim-nom.sg 2/3.s/a-tr-пачкать-cs-3sg.a...,Девочка испачкала рубашку.,00:38:14.802,00:38:22.973,В конце она совсем запуталась. И где INS?,DMP,,,,,,,
637,DMP_20220809@TK_29,ŋewʔen-jərʔ-ə rə-qametwa-w-ne-n qɬawəɬ eɬʔem-u...,жена-содержимое-ins 2/3.s/a-tr-есть-cs-3sg.a.3...,Женщина без мяса накормила мужа. (в доме не бы...,00:42:40.365,00:42:56.024,Начало car не слышно. Концовку перевела,DMP,,,,,car,мясо,жена-содержимое
638,DMP_20220809@TK_30,ŋewʔen-jərʔ-ə e-tekisɣə-k-e rə-qametwa-w-ne-n ...,жена-содержимое-ins car-мясо-car-ins 2/3.s/a-...,"Женщина без мяса (вот такая у неё особенность,...",00:43:19.982,00:43:57.303,,DMP,,,,,car,мясо,жена-содержимое
